<a href="https://colab.research.google.com/github/Renga1610/Renga1610/blob/main/Stable_Diffusion_Screen_play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
!pip install keras-cv --upgrade --quiet

In [ ]:
import keras_cv
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
from PIL import Image

# Enable mixed precision
# (only do this if you have a recent NVIDIA GPU)
keras.mixed_precision.set_global_policy("mixed_float16")

# Instantiate the Stable Diffusion model
model = keras_cv.models.StableDiffusion(jit_compile=True)

In [ ]:
import requests

url = "https://timestamp-lyrics.p.rapidapi.com/extract-lyrics"

querystring = {"name":"Blinding Lights"}

headers = {
	"X-RapidAPI-Key": "f8101922admshe96fce0a2b7e46dp1a722ejsncc4388f87300",
	"X-RapidAPI-Host": "timestamp-lyrics.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

In [ ]:
import http.client

conn = http.client.HTTPSConnection("lyrics-plus.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "f8101922admshe96fce0a2b7e46dp1a722ejsncc4388f87300",
    'X-RapidAPI-Host': "lyrics-plus.p.rapidapi.com"
    }

conn.request("GET", "/lyrics/hello/adele", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

In [ ]:
import json

parse_json = json.loads(data)

In [ ]:
lyrics = parse_json['body']['lyrics']
print("Hello by Adele Lyrics:", lyrics)

In [ ]:
lyric_list = lyrics.split("\n")

In [ ]:
interpolation_steps = 5
encoding = []

for i in range(len(lyric_list)):
  encoding.append(tf.squeeze(model.encode_text(lyric_list[i])))

interpolated_encodings = []

for i in range(len(lyric_list)-1):
  interpolated_encodings.append(tf.linspace(encoding[i], encoding[i+1], interpolation_steps))

# Show the size of the latent manifold
print(f"Encoding shape: {encoding[i].shape}")

In [ ]:
len(encoding)

In [ ]:
seed = 12345
noise = tf.random.normal((512 // 8, 512 // 8, 4), seed=seed)
images = []
for i in range(len(interpolated_encodings)):
  images.append(model.generate_image(
    interpolated_encodings[i],
    batch_size=interpolation_steps,
    diffusion_noise=noise,
))

In [ ]:
import numpy as np
images = np.array(images)

In [ ]:
images = images.reshape(-1,512,512,3)

In [ ]:
def export_as_gif(filename, images, frames_per_second=10, rubber_band=False):
    if rubber_band:
        images += images[2:-1][::-1]
    images[0].save(
        filename,
        save_all=True,
        append_images=images[1:],
        duration=1000 // frames_per_second,
        loop=0,
    )


export_as_gif(
    "hello.gif",
    [Image.fromarray(img) for img in images],
    frames_per_second=2,
    rubber_band=True,
)

In [ ]:
from IPython.display import Image as IImage
IImage("hello.gif")